In [107]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [108]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [109]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 33.7243,
	"longitude": -6.91,
	"start_date": "2014-12-01",
	"end_date": "2024-12-01",
	"hourly": "relative_humidity_2m",
	"daily": ["weather_code", "temperature_2m_mean", "precipitation_sum"]
}

In [110]:
responses = openmeteo.weather_api(url, params=params)

In [111]:
response = responses[0]

In [112]:
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 33.70825958251953°N -6.931671142578125°E
Elevation 283.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [113]:
pd.options.display.float_format = "{:.2f}".format

In [114]:
hourly = response.Hourly()
hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)

hourly_dataframe

,date,relative_humidity_2m
0,2014-12-01 00:00:00+00:00,97.40
1,2014-12-01 01:00:00+00:00,97.72
2,2014-12-01 02:00:00+00:00,98.36
3,2014-12-01 03:00:00+00:00,98.36
4,2014-12-01 04:00:00+00:00,98.36
...,...,...
87691,2024-12-01 19:00:00+00:00,76.37
87692,2024-12-01 20:00:00+00:00,79.33
87693,2024-12-01 21:00:00+00:00,78.24
87694,2024-12-01 22:00:00+00:00,77.43


In [115]:
hourly_dataframe["date"] = hourly_dataframe["date"].dt.date

In [116]:
#open wheather n'a pas des donnees d'humidite journiale donc j'ai calculer la moyenne le max et le min pour chaque jour
daily_humidity = hourly_dataframe.groupby("date")["relative_humidity_2m"].agg(
    mean="mean"
).reset_index()
daily_humidity

,date,mean
0,2014-12-01,90.43
1,2014-12-02,86.31
2,2014-12-03,92.44
3,2014-12-04,86.92
4,2014-12-05,82.78
...,...,...
3649,2024-11-27,62.71
3650,2024-11-28,47.78
3651,2024-11-29,56.02
3652,2024-11-30,45.24


In [117]:
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe["date"] = daily_dataframe["date"].dt.date
daily_dataframe

,date,weather_code,temperature_2m_mean,precipitation_sum
0,2014-12-01,51.00,13.70,0.20
1,2014-12-02,51.00,14.17,1.70
2,2014-12-03,61.00,12.76,7.70
3,2014-12-04,61.00,11.95,9.80
4,2014-12-05,51.00,11.66,0.50
...,...,...,...,...
3649,2024-11-27,3.00,18.57,0.00
3650,2024-11-28,3.00,20.25,0.00
3651,2024-11-29,3.00,18.97,0.00
3652,2024-11-30,3.00,20.37,0.00


In [118]:
#On emerge les deux dataframe :
meteo_10ans = pd.merge(
    daily_dataframe,
    daily_humidity,
    how="left",
    on="date"
)

In [119]:
meteo_10ans

,date,weather_code,temperature_2m_mean,precipitation_sum,mean
0,2014-12-01,51.00,13.70,0.20,90.43
1,2014-12-02,51.00,14.17,1.70,86.31
2,2014-12-03,61.00,12.76,7.70,92.44
3,2014-12-04,61.00,11.95,9.80,86.92
4,2014-12-05,51.00,11.66,0.50,82.78
...,...,...,...,...,...
3649,2024-11-27,3.00,18.57,0.00,62.71
3650,2024-11-28,3.00,20.25,0.00,47.78
3651,2024-11-29,3.00,18.97,0.00,56.02
3652,2024-11-30,3.00,20.37,0.00,45.24


In [120]:
meteo_10ans = meteo_10ans.drop(columns=["weather_code"])

In [121]:
meteo_10ans.rename(columns={
    "date": "Date",
    "temperature_2m_mean": "Temperature moyenne (°C)",
    "precipitation_sum": "Precipitation (mm)",
    "mean": "Humidite moyenne (%)",
}, inplace=True)

In [122]:
meteo_10ans

,Date,Temperature moyenne (°C),Precipitation (mm),Humidite moyenne (%)
0,2014-12-01,13.70,0.20,90.43
1,2014-12-02,14.17,1.70,86.31
2,2014-12-03,12.76,7.70,92.44
3,2014-12-04,11.95,9.80,86.92
4,2014-12-05,11.66,0.50,82.78
...,...,...,...,...
3649,2024-11-27,18.57,0.00,62.71
3650,2024-11-28,20.25,0.00,47.78
3651,2024-11-29,18.97,0.00,56.02
3652,2024-11-30,20.37,0.00,45.24


In [123]:
meteo_10ans.to_csv('meteo_10ans.csv',index=False)

In [124]:
meteo_10ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3654 entries, 0 to 3653
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      3654 non-null   object 
 1   Temperature moyenne (°C)  3654 non-null   float32
 2   Precipitation (mm)        3654 non-null   float32
 3   Humidite moyenne (%)      3654 non-null   float32
dtypes: float32(3), object(1)
memory usage: 71.5+ KB


In [125]:
meteo_10ans.describe()

,Temperature moyenne (°C),Precipitation (mm),Humidite moyenne (%)
count,3654.00,3654.00,3654.00
mean,17.59,1.12,74.48
std,4.91,3.55,11.80
min,6.18,0.00,19.03
25%,13.57,0.00,69.95
50%,17.49,0.00,76.97
75%,21.24,0.20,81.89
max,33.93,35.00,97.11


In [126]:
s=meteo_10ans.describe().to_dict()

In [127]:
s.keys()

dict_keys(['Temperature moyenne (°C)', 'Precipitation (mm)', 'Humidite moyenne (%)'])

In [128]:
s['Temperature moyenne (°C)']

{'count': 3654.0,
 'mean': 17.592144012451172,
 'std': 4.909886837005615,
 'min': 6.175084590911865,
 '25%': 13.574040412902832,
 '50%': 17.492582321166992,
 '75%': 21.23924970626831,
 'max': 33.9329948425293}

In [129]:
meteo_10ans['temp_rolling_mean'] = meteo_10ans['Temperature moyenne (°C)'].rolling(window=7).mean()
meteo_10ans['humidity_rolling_mean'] = meteo_10ans['Humidite moyenne (%)'].rolling(window=7).mean()

In [130]:
meteo_10ans

,Date,Temperature moyenne (°C),Precipitation (mm),Humidite moyenne (%),temp_rolling_mean,humidity_rolling_mean
0,2014-12-01,13.70,0.20,90.43,NaN,NaN
1,2014-12-02,14.17,1.70,86.31,NaN,NaN
2,2014-12-03,12.76,7.70,92.44,NaN,NaN
3,2014-12-04,11.95,9.80,86.92,NaN,NaN
4,2014-12-05,11.66,0.50,82.78,NaN,NaN
...,...,...,...,...,...,...
3649,2024-11-27,18.57,0.00,62.71,18.28,64.78
3650,2024-11-28,20.25,0.00,47.78,18.93,59.05
3651,2024-11-29,18.97,0.00,56.02,19.34,55.82
3652,2024-11-30,20.37,0.00,45.24,19.56,54.97
